## Create an agent

In [ ]:
import warnings
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from tools import *

warnings.filterwarnings("ignore", category=UserWarning, module='pydantic')

llm = ChatOllama(
    model="qwen3:8b",
    temperature=0,
    streaming=True  
) 

agent = create_react_agent(
    model=llm,  
    tools=[soma, milu, get_weather],  
    prompt="Você é um assistente útil que extrai os numeros mencionados pelo usuario para utilizar a função soma, depois faz o campati do resultado encontrado. Sempre faça o campati"  
)

resposta = agent.invoke(
    {"messages": [{"role": "user", "content": "Qual foi minha ultima msg?"}]}
)

print(resposta["messages"][-1].content)

#### Adicionando memoria

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from tools import *

#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------
#-------------Este React_Agent utiliza uma memoria de curto prazo nativa do----------------
#-------------LangGraph. O dever agora é persistir esta memoria com o Redis----------------
#------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------


llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    streaming=True  
) # llama3.1, qwen3:8b, qwen3:14b

#class WeatherResponse(BaseModel):
#    conditions: str= Field(description="a mensagem do usuario")

checkpointer = InMemorySaver()
agent = create_react_agent(
    model=llm,  
    tools=[soma, milu, get_weather],  
    prompt="Você é um assistente útil que extrai os numeros mencionados pelo usuario para utilizar a função soma, depois faz o campati do resultado encontrado. Sempre faça o campati" ,
    checkpointer=checkpointer, 
#    response_format=WeatherResponse
)

config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "Ola tudo bem?"}]},
    config  
)

#sf_response["structured_response"]
sf_response

In [ ]:
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "Me chamo john. O numero é 12 e a empresa é 32. Qual a temperatura em floripa?"}]},
    config
)

sf_response

## Criando um chatbot basico - Tutorial

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict
from enum import Enum
from typing import Optional

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

class Qualquer(str, Enum):
    alguma = "é isso"
    outra = "Ok"

class State(TypedDict):
    messages: Annotated[list, add_messages]
    stato: Optional[Qualquer]

graph_builder = StateGraph(State)

In [ ]:
from typing import Annotated, Optional
from typing_extensions import TypedDict
from enum import Enum
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

# ----------------------- Adicionando State ----------------------
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# --------------------- Adicionando Nodes ---------------------
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    streaming=True  
) # llama3.1, qwen3:8b, qwen3:14b

def chatbot(state: State) -> dict:
    """Função que invoca o modelo com as mensagens do estado."""
    return {"messages": [llm.invoke(state["messages"])]}

# O primeiro argumento é o nome único do nó
# O segundo argumento é a função ou objeto que será chamado sempre que o nó for utilizado.
graph_builder.add_node("chatbot", chatbot)

# Adicione um ponto para informar ao gráfico por onde começar seu trabalho cada vez que for executado:
graph_builder.add_edge(START, "chatbot")

# Antes de executar o grafo, precisaremos compilá-lo. Podemos fazer isso chamando o construtor de gráficos. 
# Isso cria um que podemos invocar em nosso estado.
graph = graph_builder.compile()


# --------------------- Executando o grafo ---------------------
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    stream_graph_updates(user_input)


#### Adicionando ferramentas

Forma manual de fazer

In [ ]:
from typing import Annotated, Optional
from typing_extensions import TypedDict
from langchain_tavily import TavilySearch
from enum import Enum
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
import os, json
from langchain_core.messages import ToolMessage


os.environ["TAVILY_API_KEY"] = "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"

tavily_tool = TavilySearch(max_results=2)
tools = [tavily_tool]
tool.invoke("What's a 'node' in LangGraph?")

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    streaming=True  
)

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)


llm_whith_tools = llm.bind_tools(tools)
def chatbot(state=State):
    return {"messages": [llm_whith_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)


# ---------------Criando função para executar ferramentas ------------------------
class BasicToolNode:
    """Um nó que executa as ferramentas solicitadas na última AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}
    

tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)
